# Simple Data Dictionary Generator
This Python script analyses a "source CSV" file and provides basic data analysis information for each field.

It is then possible to download and parse the summary table (in JSON format) into Excel and create a standard table from it. The goal is to develop a simple method for creating the foundation of a data dictionary for data warehouse and ETL projects.

<br>

The summary table contains the following columns:

<br>

1. **Column Name:** Name of a column (i.e. header) within the "source CSV" file.

2. **Sample Data:** A list of values, (a maximum of ten values), randomly sampled from within a column.  Each value should be separated by a semicolon.

3. **Min Value:** Minimum value within a column, only if the data type is "Number" or "Date", else for any other data type state "n-a". If all values within the column are nulls, then state "n-a".

 4. **Max Value:** Maximum value within a column, only if the data type is "Number" or "Date", else for any other data type state "n-a". If all values within the column are nulls, then state "n-a".

 5. **Data Type:** Using basic data types to analyse all of the values within a column. Data types and mappings to Python-style data types are as follows:

    - Unicode values: "Text"
    - String or text values: "Text"
    - Mixed numerical and non-numerical values: "Text"
    - Date and time values: "Date"
    - Integer numbers: "Number"
    - Floating point numbers: "Number"
    - True/False values: "Boolean"
    - Column with all NULLs: "All Nulls"

<br>

6. **Max Character Length:** If data type is "Text", the length of the shortest string value in a column, else for any other data type state "n-a". If all values within the column are nulls, then state "n-a".

7. **Min Character Length:** If data type is "Text", the length of the shortest string value in a column, else for any other data type state "n-a". If all values within the column are nulls, then state "n-a".

8. **% NULL Fields:** Percentage of values which are nulls within a column, formatted to two decimal places. If no nulls are present, state 0%.

9. **NULL field count:** Count of values which are NOT nulls within a column.

10. **Not NULL field count:** Count of values which are nulls within a column.

11. **Count of Unique Values:** Count unique values within a column. If all values are null, then state "n-a".

<br>

The code should implement the following requirements:

a) Each row of the "output" table should represent one column from the source CSV.

b) For data sets with more than 400,000 rows (this figure can be changed via the variable 'MAX_ROWS'), the analysis will be based on a random sample, equal to MAX_ROWS (default = 400k rows).

c) The "output" JSON can be easily parsed as a table.


In [1]:
import pandas as pd
import numpy as np
import json
import csv
import os

In [2]:
# Set the file paths
source_csv = "https://github.com/<replace_with_link_to_input_csv>.csv?raw=true"
output_directory = "/Users/<replace-with-your-output-path>"
output_file_name = "<replace-with-name-ofoutput-csv>.json"
output_json_file = os.path.join(output_directory, output_file_name)


In [ ]:
# Create the directory if it doesn't exist
output_directory = os.makedirs(output_directory, exist_ok=True)

# Set maximum number of rows to read from CSV
MAX_ROWS = 400000

# Initialize an empty list to store the column information
columns_info = []

In [ ]:
# Load the data into a Pandas dataframe
df = pd.read_csv(source_csv, low_memory=False)

In [ ]:
# Check if the dataframe has more than MAX_ROWS variable
if len(df) > MAX_ROWS:
    # If it does, take a random sample equal to MAX_ROWS variable
    df = df.sample(n=MAX_ROWS, random_state=1)

In [6]:
# Loop through each column in the dataframe
for col in df.columns:
    # Get the column data
    col_data = df[col]
    unique_col_data = df[col].drop_duplicates()

    # Get a sample of the column data
    sample_data = ";".join([str(val) for val in col_data.sample(n=min(10, len(col_data)), random_state=1).tolist()])
    unique_sample_data = "; ".join([str(val) for val in unique_col_data.sample(n=min(10, len(unique_col_data)), random_state=1).tolist()])
    
    # Get the data type of the column
    data_type = str(col_data.dtype)

    # Initialize the null count and unique value set
    null_count = 0
    unique_vals = set()

    # Loop through each value in the column
    for val in col_data:
        # If the value is null, increment the null count
        if pd.isnull(val):
            null_count += 1
        # If the value is not null, add it to the unique value set
        else:
            unique_vals.add(val)

    # Test if all values in coluimn are nulls
    col_all_null_test = col_data.isnull().all()

    # Calculate the percentage of nulls and the count of unique values
    null_percent = round(null_count / len(col_data) * 100, 2)
    unique_count = len(unique_vals)

# Determine data type of column
    col_drop_na = df.dropna(axis=0,subset=col)
    data_type = col_drop_na
    # data_type = col_data.dtype
    data_type_str = str(data_type)

# Map Python data types against basic set of data types as follows:

    # - Unicode values: "Text"
    # - String or text values: "Text"
    # - Mixed numeric and non-numeric values: "Text"
    # - Date and time values: "Date"
    # - Integer numbers: "Number"
    # - Floating point numbers: "Number"
    # - True/False values: "Boolean"
    # - Column with all NULLs: "All Nulls

    if data_type == np.dtype('O'):
        # Column contains strings or mixed numerical and non-numerical values
        char_lengths = [len(str(val)) for val in col_data if not pd.isnull(val)]
        data_type_str = "Text"
        min_val = "n-a"
        max_val = "n-a"
        min_char_length = min(char_lengths)
        max_char_length = max(char_lengths)

    elif np.issubdtype(data_type, np.number):
        # Column contains numeric values
        data_type_str = "Number"
        min_val = str(col_data.min())
        max_val = str(col_data.max())
        min_char_length = "n-a"
        max_char_length = "n-a"

    elif data_type == 'bool':
        # Column contains True/False values
        data_type_str = "Boolean"
        min_val = "n-a"
        max_val = "n-a"
        min_char_length = "n-a"
        max_char_length = "n-a"

    elif np.issubdtype(data_type, np.datetime64):
        # Column contains date/time values
        data_type_str =  "Date/Time"
        min_val = col_data.min().strftime("%Y-%m-%d %H:%M:%S")
        max_val = col_data.max().strftime("%Y-%m-%d %H:%M:%S")
        min_char_length = "n-a"
        max_char_length = "n-a"

    else:
        # Column contains unknown data type
        data_type_str = "Unknown"
        min_val = "n-a"
        max_val = "n-a"
        min_char_length = "n-a"
        max_char_length = "n-a"


    # Create a dictionary with the column information
    col_info = {
        "Ordinal Position (Zero based)": df.columns.get_loc(col),
        "Column Name": col,
        "Sample Data": unique_sample_data,
        "Min Value": min_val,
        "Max Value": max_val,
        "Data Type": data_type_str,
        "Max Character Length": max_char_length,
        "Min Character Length": min_char_length,
        "Column All Nulls": "All null" if col_all_null_test == True else "No",
        "% NULL fields": f"{null_percent:.2f}%",
        "NULL field count": null_count,
        "Count of Unique Values": unique_count }
    
    # print(col_info)
    # Append col_info iteration to main col_information list
    columns_info.append(col_info.copy())

# Convert list of dictionaries to JSON file
with open(output_json_file, 'w') as json_file:
    json.dump(columns_info, json_file, ensure_ascii=False, indent=4, separators=(',',': '))

json_file.close()
       
